In [3]:
import sys
import time
import math
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy.special import logit
from scipy.stats import norm

import tensorflow as tf
from keras import layers, models, datasets

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR, LambdaLR
import torch.autograd.profiler as profiler

from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PowerTransformer
from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.linear_model import LogisticRegression

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
def calculate_metrics(model, data_tensor, labels_tensor, batch_size=1024, num_features=22):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for start_idx in range(0, len(data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(data_tensor))
            inputs = data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = labels_tensor[start_idx:end_idx]

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [5]:
class CustomDataLoader:
    def __init__(self, features, labels, validation_size=0.2, random_state=42, classification=True):        
        if validation_size > 0.0:
            stratify = labels if classification else None
            train_data, val_data, train_labels, val_labels = train_test_split(
                features, labels, test_size=validation_size, stratify=stratify, random_state=random_state
            )
            
            self.val_data_tensor = torch.tensor(val_data).float().to(device)
            
            if classification:
                self.val_labels_tensor = torch.tensor(val_labels).long().to(device)

            else:
                self.val_labels_tensor =torch.tensor(val_labels).float().to(device)
        else:
            train_data, train_labels = features, labels
            self.val_data_tensor, self.val_labels_tensor = None, None
        
        self.train_data_tensor = torch.tensor(train_data).float().to(device)

        if classification:
            self.train_labels_tensor = torch.tensor(train_labels).long().to(device)
        else:
            self.train_labels_tensor = torch.tensor(train_labels).float().to(device)

        torch.manual_seed(random_state)
        indices = torch.randperm(len(self.train_data_tensor))

        self.train_data_tensor = self.train_data_tensor[indices]
        self.train_labels_tensor = self.train_labels_tensor[indices]

In [6]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size=1024, num_features=22, early_stopping_patience=10):
    best_val_loss = float('inf')
    best_epoch = 0
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        i = 0
        total_loss = 0
        num_items = 0

        for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
            inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = custom_train_loader.train_labels_tensor[start_idx:end_idx]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels, model)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * len(labels)
            total_loss += loss.item() * len(labels)
            num_items += len(labels)

            i += 1

        if epoch % 10 == 0:
            for param_group in optimizer.param_groups:
                print("Learning Rate:", param_group['lr'])

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for start_idx in range(0, len(custom_train_loader.val_data_tensor), batch_size):
                    end_idx = min(start_idx + batch_size, len(custom_train_loader.val_data_tensor))
                    val_inputs = custom_train_loader.val_data_tensor[start_idx:end_idx].view(-1, num_features)
                    val_labels = custom_train_loader.val_labels_tensor[start_idx:end_idx]
    
                    val_outputs = model(val_inputs)
                    val_loss += criterion.regular_loss(val_outputs, val_labels).item() * len(val_labels)
    
            avg_train_loss = running_loss / len(custom_train_loader.train_data_tensor)
            avg_val_loss = val_loss / len(custom_train_loader.val_data_tensor)
    
            train_accuracy, train_f1 = calculate_metrics(model, custom_train_loader.train_data_tensor, custom_train_loader.train_labels_tensor, batch_size, num_features)
            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.val_data_tensor, custom_train_loader.val_labels_tensor, batch_size, num_features)
    
            print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
            print(f'Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1}')
            print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')
            print()
            
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_epoch = epoch + 1
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= early_stopping_patience:
                    print(f'Early stopping triggered after {epoch + 1} epochs.')
                    print(f'Best Validation Loss: {best_val_loss} from Epoch {best_epoch}')
                    break

    if patience_counter < early_stopping_patience:
        print(f'Best Validation Loss after {num_epochs} epochs: {best_val_loss} from Epoch {best_epoch}')

In [7]:
data_dl = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
data_og = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

data_dl = data_dl.drop(["id"], axis=1)

median_emp_length = data_og['person_emp_length'].median()
median_int_rate = data_og['loan_int_rate'].median()

data_dl['source'] = 0
data_og['source'] = 1

data = pd.concat([data_dl, data_og], ignore_index=True)

data['person_emp_length_missing'] = data['person_emp_length'].isna().astype(int)
data['loan_int_rate_missing'] = data['loan_int_rate'].isna().astype(int)

data['person_emp_length'] = data['person_emp_length'].fillna(median_emp_length)
data['loan_int_rate'] = data['loan_int_rate'].fillna(median_int_rate)

grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
data['loan_grade'] = data['loan_grade'].map(grade_mapping)

purpose_mapping = {
    'DEBTCONSOLIDATION': 1,
    'HOMEIMPROVEMENT': 2,
    'MEDICAL': 3,
    'PERSONAL': 4,
    'EDUCATION': 5,
    'VENTURE': 6
}
data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

home_ownership_mapping = {
    'OWN': 1,
    'MORTGAGE': 2,
    'OTHER': 3,
    'RENT': 4
}
data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

X = data.drop(["loan_status"], axis=1)
X = pd.get_dummies(X, drop_first=True)
y = data["loan_status"]

column_to_log = [
    'person_age',
    'person_income',
]

column_to_sqrt = [
    'person_emp_length',
    'loan_percent_income',
]

for col in column_to_log:
    if (X[col] <= 0).any():
        print(f"Column '{col}' contains non-positive values. Adding 1 to avoid log of non-positive numbers.")
        X[col] = np.log(X[col] + 1)
    else:
        X[col] = np.log(X[col])

for col in column_to_sqrt:
    if (X[col] < 0).any():
        print(f"Column '{col}' contains negative values. Setting negative values to NaN before applying sqrt.")
        X[col] = np.sqrt(X[col].clip(lower=0))
    else:
        X[col] = np.sqrt(X[col])

print(data.isnull().sum())
print(X.columns)
print(X.shape, y.shape)
print(X.columns.get_loc('source'))

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
source                        0
person_emp_length_missing     0
loan_int_rate_missing         0
dtype: int64
Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'source', 'person_emp_length_missing', 'loan_int_rate_missing',
       'cb_person_default_on_file_Y'],
      dtype='object')
(91226, 14) (91226,)
10


In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

print(X.shape, y.shape)

rf = RandomForestClassifier()
rf.fit(X, y)

importances = rf.feature_importances_

feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
X = X[feature_importances['Feature']]

print("Feature importances (from most important to least important):")
print(feature_importances)
print(X.columns)

(91226, 14) (91226,)
Feature importances (from most important to least important):
                        Feature  Importance
8           loan_percent_income    0.221108
1                 person_income    0.131752
7                 loan_int_rate    0.121694
5                    loan_grade    0.117863
2         person_home_ownership    0.093484
6                     loan_amnt    0.074997
4                   loan_intent    0.070340
3             person_emp_length    0.061402
0                    person_age    0.044438
9    cb_person_cred_hist_length    0.035300
13  cb_person_default_on_file_Y    0.014757
10                       source    0.009103
12        loan_int_rate_missing    0.001924
11    person_emp_length_missing    0.001839
Index(['loan_percent_income', 'person_income', 'loan_int_rate', 'loan_grade',
       'person_home_ownership', 'loan_amnt', 'loan_intent',
       'person_emp_length', 'person_age', 'cb_person_cred_hist_length',
       'cb_person_default_on_file_Y', 'source',

In [8]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier

print(X.shape, y.shape)

gbc = GradientBoostingClassifier(max_depth=5)
gbc.fit(X, y)

importances = gbc.feature_importances_

feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

X = X[feature_importances['Feature']]

print("Feature importances (from most important to least important):")
print(feature_importances)
print(X.columns)
print(gbc.train_score_)

(91226, 14) (91226,)
Feature importances (from most important to least important):
                        Feature  Importance
0           loan_percent_income    0.319132
3                    loan_grade    0.251763
4         person_home_ownership    0.184154
6                   loan_intent    0.088150
1                 person_income    0.087999
7             person_emp_length    0.041101
2                 loan_int_rate    0.013917
8                    person_age    0.005955
11                       source    0.004103
5                     loan_amnt    0.002430
9    cb_person_cred_hist_length    0.000677
13    person_emp_length_missing    0.000494
10  cb_person_default_on_file_Y    0.000085
12        loan_int_rate_missing    0.000043
Index(['loan_percent_income', 'loan_grade', 'person_home_ownership',
       'loan_intent', 'person_income', 'person_emp_length', 'loan_int_rate',
       'person_age', 'source', 'loan_amnt', 'cb_person_cred_hist_length',
       'person_emp_length_missing', '

In [8]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(x_scaled.shape)

(91226, 14)


In [9]:
feature_means = x_scaled.mean(axis=0)
feature_variances = x_scaled.var(axis=0)
feature_mins = x_scaled.min(axis=0)
feature_maxs = x_scaled.max(axis=0)

feature_stats_scaled_full = pd.DataFrame({
    'Mean': feature_means,
    'Variance': feature_variances,
    'Min': feature_mins,
    'Max': feature_maxs
})

print("Mean, Variance, Min, and Max of Scaled Features:")
print(feature_stats_scaled_full)

Mean, Variance, Min, and Max of Scaled Features:
            Mean  Variance       Min        Max
0  -3.289997e-16       1.0 -1.552712   8.591255
1  -1.420680e-16       1.0 -5.315235   9.344027
2  -1.370832e-17       1.0 -1.810229   0.945472
3   3.987875e-17       1.0 -1.859550   8.913061
4   6.153166e-17       1.0 -1.591801   1.383254
5  -1.183900e-17       1.0 -4.464004   1.025387
6  -1.333446e-16       1.0 -1.513249   4.385625
7   9.327889e-16       1.0 -1.759487   4.065809
8  -5.358707e-16       1.0 -3.245038   4.413749
9   6.480297e-17       1.0 -0.943500   5.989958
10 -1.944089e-16       1.0 -0.745361   1.341632
11 -4.610980e-17       1.0 -0.099539  10.046317
12  6.729539e-17       1.0 -0.188056   5.317578
13  2.928596e-17       1.0 -0.433778   2.305326


In [10]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, f1_lambda, f2_lambda, l1_lambda, l2_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.f1_lambda = f1_lambda
        self.f2_lambda = f2_lambda
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def forward(self, outputs, labels, model): 
        f1_loss = 0.0
        f2_loss = 0.0
        l1_loss = 0.0
        l2_loss = 0.0

        for name, module in model.named_modules():
            if isinstance(module, CustomActivation):
                f1_loss += (module.a ** 2).sum() + (module.b ** 2).sum()
                f2_loss += ((module.a - module.b) ** 2).sum()

            if isinstance(module, nn.Linear):
                l1_loss += torch.norm(module.weight, 1)
                l2_loss += torch.norm(module.weight, 2) ** 2

        total_loss = (self.criterion(outputs, labels)
                      + self.f1_lambda * f1_loss
                      + self.f2_lambda * f2_loss
                      + self.l1_lambda * l1_loss
                      + self.l2_lambda * l2_loss)

        return total_loss

    def regular_loss(self, outputs, labels):
        return self.criterion(outputs, labels)

In [11]:
class CustomActivation(nn.Module):
    def __init__(self, num_features, num_control_points, bias_tensor, init_identity=False):
        super(CustomActivation, self).__init__()
        self.a = nn.Parameter(torch.zeros(num_features, num_control_points))
        self.b = nn.Parameter(torch.zeros(num_features, num_control_points))
        
        self.local_bias = nn.Parameter(torch.zeros(num_features, num_control_points))
        self.global_bias = nn.Parameter(torch.zeros(1, num_features))
        # global_bias may not even be needed

        with torch.no_grad():
            repeated_bias = bias_tensor.repeat(num_features // bias_tensor.shape[0], 1)
            self.local_bias.copy_(repeated_bias)

            if init_identity:
                middle_index = num_control_points // 2
                self.a[:, middle_index] = 1.0
                self.b[:, middle_index] = 1.0

    def forward(self, x):
        x = x.unsqueeze(-1) + self.local_bias
        x = torch.where(x < 0, self.a * x, self.b * x)
        return x.sum(dim=-1) + self.global_bias

In [12]:
class CustomLinear(nn.Module):
    def __init__(self, num_features, num_outputs, init_identity=False):
        super(CustomLinear, self).__init__()
        
        if init_identity and num_features != num_outputs:
            raise ValueError("For identity initialization, num_features must equal num_outputs.")

        self.linear = nn.Linear(num_features, num_outputs, bias=True)
        
        with torch.no_grad():
            self.linear.bias.zero_()

            if init_identity:
                self.linear.weight.copy_(torch.eye(num_features, num_outputs))
            else:
                self.linear.weight.zero_()
                            
    def forward(self, x):
        return self.linear(x)

In [13]:
class TabularDenseNet(nn.Module):
    def __init__(self, input_size, output_size, num_control_points, num_layers, corr_comb_indices):
        super(TabularDenseNet, self).__init__()
        self.layers = nn.ModuleList()
        self.corr_comb_indices = corr_comb_indices

        quantiles = np.quantile(x_scaled, q=np.linspace(0, 1, num_control_points), axis=0).T
        bias_tensor = torch.tensor(quantiles)
        
        index_expanded = corr_comb_indices.unsqueeze(1).expand(-1, bias_tensor.size(1))
        bias_tensor_comb = torch.gather(bias_tensor, dim=0, index=index_expanded)
        
        if num_layers % 2 == 1:
            self.layers.append(CustomActivation(input_size, num_control_points, bias_tensor_comb, init_identity=True))
            num_layers -= 1
            input_size *= 2
            
        for i in range(num_layers):
            if i % 2 == 0:
                self.layers.append(CustomLinear(input_size, input_size, init_identity=True))
            else:
                self.layers.append(CustomActivation(input_size, num_control_points, bias_tensor_comb, init_identity=True))

            input_size *= 2
            
        self.final = CustomLinear(input_size, output_size, init_identity=False)
        
    def forward(self, x):
        combinations_tensor = x[:, self.corr_comb_indices]
        outputs = [combinations_tensor]

        for layer in self.layers:
            concatenated_outputs = torch.cat(outputs, dim=-1)
            outputs.append(layer(concatenated_outputs))

        concatenated_outputs = torch.cat(outputs, dim=-1)
        x = self.final(concatenated_outputs)
        return x

In [14]:
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, validation_size=0.2, random_state=0, classification=True)
print(custom_train_loader.train_data_tensor.shape)

torch.Size([72980, 14])


In [15]:
interaction_order = 11
x = custom_train_loader.train_data_tensor
batch_size = x.size(0)
num_features = x.size(1)

comb_indices = list(itertools.combinations(range(num_features), interaction_order))
comb_indices = torch.tensor(comb_indices, dtype=torch.long)
corr_comb_indices = comb_indices[0]

print(corr_comb_indices.shape)
corr_comb_indices[0] = 13
corr_comb_indices[1] = 12
corr_comb_indices[2] = 11
corr_comb_indices[3] = 10
corr_comb_indices[4] = 9
corr_comb_indices[5] = 8
corr_comb_indices[6] = 7
corr_comb_indices[7] = 6
corr_comb_indices[8] = 5
corr_comb_indices[9] = 4
corr_comb_indices[10] = 3
print(corr_comb_indices)

torch.Size([11])
tensor([13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3])


In [16]:
models = []

In [17]:
feature_tensor = torch.empty((custom_train_loader.train_data_tensor.size(0), 0))
print(feature_tensor.shape)

torch.Size([72980, 0])


In [19]:
num_og_features = 14
num_features = interaction_order
num_classes = 2
num_control_points = 15
num_epochs = 100
batch_size = 7298

In [21]:
import torch
import torch.optim as optim
import torch.nn as nn
import time

# Assume device is set to 'cuda'
device = torch.device('cuda')

# Initialize models, optimizers, schedulers, criteria, and streams
models = []
optimizers = []
schedulers = []
criterions = []
streams = []
start_times = []

num_models = 2  # Number of models you want to train concurrently

for _ in range(num_models):
    start_time = time.time()
    model = TabularDenseNet(num_features, num_classes, num_control_points, 4, corr_comb_indices).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001 * 5)
    scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)
    criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0001 * 0.0, 0.0001 * 0.0)
    stream = torch.cuda.Stream(device=device)  # Create a new CUDA stream

    models.append(model)
    optimizers.append(optimizer)
    schedulers.append(scheduler)
    criterions.append(criterion)
    streams.append(stream)
    start_times.append(start_time)

# Modify the evaluate_model function to accept a stream parameter
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size=1024, num_features=22, early_stopping_patience=10, stream=None):
    with torch.cuda.stream(stream):
        best_val_loss = float('inf')
        best_epoch = 0
        patience_counter = 0

        for epoch in range(num_epochs):
            running_loss = 0.0
            model.train()
            total_loss = 0
            num_items = 0

            for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
                end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
                inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features).to(device, non_blocking=True)
                labels = custom_train_loader.train_labels_tensor[start_idx:end_idx].to(device, non_blocking=True)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels, model)
                loss.backward()
                optimizer.step()
                scheduler.step()
                running_loss += loss.item() * len(labels)
                total_loss += loss.item() * len(labels)
                num_items += len(labels)

            if epoch % 10 == 0:
                for param_group in optimizer.param_groups:
                    print("Learning Rate:", param_group['lr'])

                model.eval()
                val_loss = 0.0
                with torch.no_grad():
                    for start_idx in range(0, len(custom_train_loader.val_data_tensor), batch_size):
                        end_idx = min(start_idx + batch_size, len(custom_train_loader.val_data_tensor))
                        val_inputs = custom_train_loader.val_data_tensor[start_idx:end_idx].view(-1, num_features).to(device, non_blocking=True)
                        val_labels = custom_train_loader.val_labels_tensor[start_idx:end_idx].to(device, non_blocking=True)
        
                        val_outputs = model(val_inputs)
                        val_loss += criterion.regular_loss(val_outputs, val_labels).item() * len(val_labels)
        
                avg_train_loss = running_loss / len(custom_train_loader.train_data_tensor)
                avg_val_loss = val_loss / len(custom_train_loader.val_data_tensor)
        
                train_accuracy, train_f1 = calculate_metrics(model, custom_train_loader.train_data_tensor, custom_train_loader.train_labels_tensor, batch_size, num_features)
                val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.val_data_tensor, custom_train_loader.val_labels_tensor, batch_size, num_features)
        
                print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
                print(f'Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1}')
                print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')
                print()
                
                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    best_epoch = epoch + 1
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= early_stopping_patience:
                        print(f'Early stopping triggered after {epoch + 1} epochs.')
                        print(f'Best Validation Loss: {best_val_loss} from Epoch {best_epoch}')
                        break

        if patience_counter < early_stopping_patience:
            print(f'Best Validation Loss after {num_epochs} epochs: {best_val_loss} from Epoch {best_epoch}')

# Start training each model in its own CUDA stream
for idx in range(num_models):
    model = models[idx]
    optimizer = optimizers[idx]
    scheduler = schedulers[idx]
    criterion = criterions[idx]
    stream = streams[idx]
    start_time = start_times[idx]

    # Enqueue the training function in the model's CUDA stream
    with torch.cuda.stream(stream):
        evaluate_model(
            model,
            custom_train_loader,
            criterion,
            optimizer,
            num_epochs,
            scheduler,
            batch_size,
            num_og_features,
            early_stopping_patience=100,
            stream=stream
        )
        print(f"Execution time: {(time.time() - start_time):.6f} seconds")

# No synchronization here; models will train concurrently


Learning Rate: 0.0005
Epoch 1, Training Loss: 0.6636582732200622, Validation Loss: 0.5848417629945256
Training Accuracy: 0.8025897506166073, Training F1 Score: 0.7872408551911488
Validation Accuracy: 0.8042310643428696, Validation F1 Score: 0.7878074382353282

here
Learning Rate: 0.004995002249400106
Epoch 11, Training Loss: 0.33306731283664703, Validation Loss: 0.32905775370676765
Training Accuracy: 0.8691422307481502, Training F1 Score: 0.8490225150045545
Validation Accuracy: 0.8723555847857064, Validation F1 Score: 0.8544899989614687

Learning Rate: 0.00494529867378049
Epoch 21, Training Loss: 0.3097297430038452, Validation Loss: 0.30558189643875855
Training Accuracy: 0.8795834475198685, Training F1 Score: 0.8625171478285869
Validation Accuracy: 0.8823303737805547, Validation F1 Score: 0.8669524068588477

Learning Rate: 0.004896089681607692
Epoch 31, Training Loss: 0.295952108502388, Validation Loss: 0.29243194667848466
Training Accuracy: 0.8882159495752261, Training F1 Score: 0.873

In [26]:
import torch
import torch.optim as optim
import torch.nn as nn
import time
from threading import Thread

# Assume device is set to 'cuda'
device = torch.device('cuda')

# Initialize models, optimizers, schedulers, criteria, and start times
models = []
optimizers = []
schedulers = []
criterions = []
start_times = []

num_models = 100 # Number of models you want to train concurrently

def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.9999 ** (step - num_step_threshold)


for _ in range(num_models):
    start_time = time.time()
    model = TabularDenseNet(num_features, num_classes, num_control_points, 4, corr_comb_indices).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001 * 5)
    scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)
    criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0001 * 0.0, 0.0001 * 0.0)
    
    models.append(model)
    optimizers.append(optimizer)
    schedulers.append(scheduler)
    criterions.append(criterion)
    start_times.append(start_time)

# Modify the evaluate_model function if needed (we'll keep it as is for now)
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size=1024, num_features=22, early_stopping_patience=10):
    best_val_loss = float('inf')
    best_epoch = 0
    patience_counter = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        total_loss = 0
        num_items = 0

        for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
            inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features).to(device, non_blocking=True)
            labels = custom_train_loader.train_labels_tensor[start_idx:end_idx].to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels, model)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * len(labels)
            total_loss += loss.item() * len(labels)
            num_items += len(labels)

        if epoch % 10 == 0:
            for param_group in optimizer.param_groups:
                print("Learning Rate:", param_group['lr'])

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for start_idx in range(0, len(custom_train_loader.val_data_tensor), batch_size):
                    end_idx = min(start_idx + batch_size, len(custom_train_loader.val_data_tensor))
                    val_inputs = custom_train_loader.val_data_tensor[start_idx:end_idx].view(-1, num_features).to(device, non_blocking=True)
                    val_labels = custom_train_loader.val_labels_tensor[start_idx:end_idx].to(device, non_blocking=True)
        
                    val_outputs = model(val_inputs)
                    val_loss += criterion.regular_loss(val_outputs, val_labels).item() * len(val_labels)
        
            avg_train_loss = running_loss / len(custom_train_loader.train_data_tensor)
            avg_val_loss = val_loss / len(custom_train_loader.val_data_tensor)
        
            train_accuracy, train_f1 = calculate_metrics(model, custom_train_loader.train_data_tensor, custom_train_loader.train_labels_tensor, batch_size, num_features)
            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.val_data_tensor, custom_train_loader.val_labels_tensor, batch_size, num_features)
        
            print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
            print(f'Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1}')
            print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')
            print()
            
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_epoch = epoch + 1
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= early_stopping_patience:
                    print(f'Early stopping triggered after {epoch + 1} epochs.')
                    print(f'Best Validation Loss: {best_val_loss} from Epoch {best_epoch}')
                    break

    if patience_counter < early_stopping_patience:
        print(f'Best Validation Loss after {num_epochs} epochs: {best_val_loss} from Epoch {best_epoch}')

# Define a function to train a model in a separate thread
def train_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size, num_features, early_stopping_patience, start_time):
    evaluate_model(
        model,
        custom_train_loader,
        criterion,
        optimizer,
        num_epochs,
        scheduler,
        batch_size,
        num_features,
        early_stopping_patience=early_stopping_patience
    )
    print(f"Execution time: {(time.time() - start_time):.6f} seconds")

# Start training each model in its own thread
threads = []

for idx in range(num_models):
    model = models[idx]
    optimizer = optimizers[idx]
    scheduler = schedulers[idx]
    criterion = criterions[idx]
    start_time = start_times[idx]

    thread = Thread(target=train_model, args=(
        model,
        custom_train_loader,
        criterion,
        optimizer,
        num_epochs,
        scheduler,
        batch_size,
        num_og_features,
        100,
        start_time
    ))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

print("something")
# Models have been trained concurrently


Exception in thread Thread-65 (train_model):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-105 (train_model):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    _threading_Thread_run(self)
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_30/964234102.py", line 111, in train_model
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_30/964234102.py", line 111, in train_model
  File "/tmp/ipykernel_30/964234102

Learning Rate: 0.0005
Epoch 1, Training Loss: 0.6647405982017517, Validation Loss: 0.5879107658491707
Training Accuracy: 0.7906412715812552, Training F1 Score: 0.8026970990175661
Validation Accuracy: 0.7935985969527568, Validation F1 Score: 0.8050613753268969

Learning Rate:Learning Rate: 0.0005
Learning Rate: 0.0005
 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate: 0.0005
Learning Rate

In [31]:
torch.cuda.empty_cache()

In [19]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.9999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, num_control_points, 4, corr_comb_indices).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 5)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0001 * 0.0, 0.0001 * 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size, num_og_features, early_stopping_patience=100)

models.append(model)
print(f"Execution time: {(time.time() - start_time):.6f} seconds")

Learning Rate: 0.0005
Epoch 1, Training Loss: 0.6636582732200622, Validation Loss: 0.5848417629945256
Training Accuracy: 0.8025897506166073, Training F1 Score: 0.7872408551911488
Validation Accuracy: 0.8042310643428696, Validation F1 Score: 0.7878074382353282

here
Learning Rate: 0.004995002249400106
Epoch 11, Training Loss: 0.33306731283664703, Validation Loss: 0.32905775370676765
Training Accuracy: 0.8691422307481502, Training F1 Score: 0.8490225150045545
Validation Accuracy: 0.8723555847857064, Validation F1 Score: 0.8544899989614687

Learning Rate: 0.00494529867378049
Epoch 21, Training Loss: 0.3097297430038452, Validation Loss: 0.30558189643875855
Training Accuracy: 0.8795834475198685, Training F1 Score: 0.8625171478285869
Validation Accuracy: 0.8823303737805547, Validation F1 Score: 0.8669524068588477

Learning Rate: 0.004896089681607692
Epoch 31, Training Loss: 0.295952108502388, Validation Loss: 0.29243194667848466
Training Accuracy: 0.8882159495752261, Training F1 Score: 0.873

KeyboardInterrupt: 

In [ ]:
for name, param in model.named_parameters():
    print(name)
    print(param)

In [ ]:
num_og_features = 14
num_features = interaction_order
num_classes = 2
num_control_points = 15
num_epochs = 1000
batch_size = 7298

def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.9999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, num_control_points, 4, corr_comb_indices).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01 * 0.02 * 10)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0001 * 0.0, 0.0001 * 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size, num_og_features, early_stopping_patience=100)

models.append(model)
print(f"Execution time: {(time.time() - start_time):.6f} seconds")

In [ ]:
data = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')

data = data.drop(["id"], axis=1)
data['source'] = 0

grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
data['loan_grade'] = data['loan_grade'].map(grade_mapping)

purpose_mapping = {
    'DEBTCONSOLIDATION': 1,
    'HOMEIMPROVEMENT': 2,
    'MEDICAL': 3,
    'PERSONAL': 4,
    'EDUCATION': 5,
    'VENTURE': 6
}
data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

home_ownership_mapping = {
    'OWN': 1,
    'MORTGAGE': 2,
    'OTHER': 3,
    'RENT': 4
}
data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

print(data.columns)
print(data.isnull().sum())

X = data.drop([], axis=1)

X = pd.get_dummies(X, drop_first=True)

column_to_log = [
    'person_age',
    'person_income',
]

column_to_sqrt = [
    'person_emp_length',
    'loan_percent_income',
]

for col in column_to_log:
    if (X[col] <= 0).any():
        print(f"Column '{col}' contains non-positive values. Adding 1 to avoid log of non-positive numbers.")
        X[col] = np.log(X[col] + 1)
    else:
        X[col] = np.log(X[col])

for col in column_to_sqrt:
    if (X[col] < 0).any():
        print(f"Column '{col}' contains negative values. Setting negative values to NaN before applying sqrt.")
        X[col] = np.sqrt(X[col].clip(lower=0))
    else:
        X[col] = np.sqrt(X[col])

print(X.shape)
print(X.columns)

In [ ]:
print(x_scaled)

In [ ]:
print(X.shape)
X_scaled_test = x_scaler.transform(X)
print(X_scaled_test.shape)
print(X_scaled_test)

In [ ]:
X_scaled_test_tensor = torch.tensor(X_scaled_test).float().to(device)
outputs = models[-1](X_scaled_test_tensor)
print(outputs)

In [ ]:
probabilities = F.softmax(outputs, dim=1)
print(probabilities)

In [ ]:
positive_class_probs = probabilities[:, 1]
print(positive_class_probs)

In [ ]:
import pandas as pd

test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
ids = test_df['id']

positive_class_probs = positive_class_probs.cpu().detach().numpy()

submission_df = pd.DataFrame({
    'id': ids,
    'loan_status': positive_class_probs
})

submission_df.to_csv('submission.csv', index=False)
print("Submission file created successfully.")